In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))

import pandas as pd
from pycaret.classification import ClassificationExperiment
import src.utils as utils
import src.calcEMA as calc_utils
import src.myenv as myenv
from datetime import datetime

from itertools import combinations
import plotly.express as px

<built-in method __dir__ of sqlite3.Connection object at 0x7f356908c940>


In [2]:
# Variables
# ETCUSDT BTCUSDT
# symbol = 'ETHUSDT'
symbol = 'BTCUSDT'
interval = '1h'
# lightgbm  xgboost lr et rf
estimator = 'knn'
_compare_models = False

start_train_date = '2022-01-01'  # train < and test >=
start_test_date = '2023-01-01'  # train < and test >=

stop_loss = 1.0
# regression_times = 0  # 24 * 30 * 2  # horas
times_regression_PnL = 24
normalize = True
use_gpu = False
tune_model = False
apply_combination_features = False

### Metadata

<code>
Field Name - Description</br>
open_time - Kline Open time in unix time format</br>
open - Open Price</br>
high - High Price</br>
low	- Low Price</br>
close	- Close Price</br>
volume - Volume</br>
close_time - Kline Close time in unix time format</br>
quote_volume - Quote Asset Volume</br>
count	- Number of Trades</br>
taker_buy_volume - Taker buy base asset volume during this period</br>
taker_buy_quote_volume - Taker buy quote asset volume during this period</br>
ignore - Ignore</br>
</code>

In [3]:
cols = myenv.all_klines_cols.copy()
cols.remove('ignore')
data = utils.get_data(symbol=symbol, save_database=False, interval=interval, tail=-1, columns=cols, parse_dates=True, start_date=start_train_date)
data = data[data['open_time'] >= start_train_date]
data = utils.parse_type_fields(data, parse_dates=True)
data = utils.adjust_index(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15710 entries, 2022-01-01 00:00:00 to 2023-10-17 14:00:00
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     15710 non-null  datetime64[ns]
 1   open                          15710 non-null  float32       
 2   high                          15710 non-null  float32       
 3   low                           15710 non-null  float32       
 4   close                         15710 non-null  float32       
 5   volume                        15710 non-null  float32       
 6   close_time                    15710 non-null  int64         
 7   quote_asset_volume            15710 non-null  float32       
 8   number_of_trades              15710 non-null  int16         
 9   taker_buy_base_asset_volume   15710 non-null  float32       
 10  taker_buy_quote_asset_volume  15710 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol
ix_open_time,,,,,,,,,,,,
2022-01-01 00:00:00,2022-01-01 00:00:00,46216.929688,46731.390625,46208.371094,46656.128906,1503.330933,1640998799999,69879984.00,-26928,806.065369,37462156.00,BTCUSDT
2022-01-01 01:00:00,2022-01-01 01:00:00,46656.140625,46949.988281,46574.058594,46778.140625,943.815369,1641002399999,44127148.00,31872,491.180664,22963272.00,BTCUSDT
2022-01-01 02:00:00,2022-01-01 02:00:00,46778.140625,46928.941406,46721.960938,46811.769531,485.168610,1641005999999,22720670.00,24364,206.351135,9662607.00,BTCUSDT
2022-01-01 03:00:00,2022-01-01 03:00:00,46811.769531,46916.628906,46760.121094,46813.199219,562.889709,1641009599999,26363260.00,19882,248.282120,11629376.00,BTCUSDT
2022-01-01 04:00:00,2022-01-01 04:00:00,46813.210938,46887.328125,46591.230469,46711.050781,861.883911,1641013199999,40272044.00,23357,397.535645,18575402.00,BTCUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-17 10:00:00,2023-10-17 10:00:00,28446.789062,28609.380859,28409.269531,28506.130859,1691.276367,1697540399999,48207104.00,7645,938.108948,26737162.00,BTCUSDT
2023-10-17 11:00:00,2023-10-17 11:00:00,28506.130859,28527.380859,28417.550781,28422.630859,1074.049072,1697543999999,30576014.00,-20184,480.409637,13676177.00,BTCUSDT
2023-10-17 12:00:00,2023-10-17 12:00:00,28422.630859,28471.300781,28180.000000,28218.919922,2213.095703,1697547599999,62722552.00,15982,935.105774,26500074.00,BTCUSDT


In [4]:
data = calc_utils.calc_RSI(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15710 entries, 2022-01-01 00:00:00 to 2023-10-17 14:00:00
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     15710 non-null  datetime64[ns]
 1   open                          15710 non-null  float32       
 2   high                          15710 non-null  float32       
 3   low                           15710 non-null  float32       
 4   close                         15710 non-null  float32       
 5   volume                        15710 non-null  float32       
 6   close_time                    15710 non-null  int64         
 7   quote_asset_volume            15710 non-null  float32       
 8   number_of_trades              15710 non-null  int16         
 9   taker_buy_base_asset_volume   15710 non-null  float32       
 10  taker_buy_quote_asset_volume  15710 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi
ix_open_time,,,,,,,,,,,,,
2022-01-01 00:00:00,2022-01-01 00:00:00,46216.929688,46731.390625,46208.371094,46656.128906,1503.330933,1640998799999,69879984.00,-26928,806.065369,37462156.00,BTCUSDT,NaN
2022-01-01 01:00:00,2022-01-01 01:00:00,46656.140625,46949.988281,46574.058594,46778.140625,943.815369,1641002399999,44127148.00,31872,491.180664,22963272.00,BTCUSDT,NaN
2022-01-01 02:00:00,2022-01-01 02:00:00,46778.140625,46928.941406,46721.960938,46811.769531,485.168610,1641005999999,22720670.00,24364,206.351135,9662607.00,BTCUSDT,NaN
2022-01-01 03:00:00,2022-01-01 03:00:00,46811.769531,46916.628906,46760.121094,46813.199219,562.889709,1641009599999,26363260.00,19882,248.282120,11629376.00,BTCUSDT,NaN
2022-01-01 04:00:00,2022-01-01 04:00:00,46813.210938,46887.328125,46591.230469,46711.050781,861.883911,1641013199999,40272044.00,23357,397.535645,18575402.00,BTCUSDT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-17 10:00:00,2023-10-17 10:00:00,28446.789062,28609.380859,28409.269531,28506.130859,1691.276367,1697540399999,48207104.00,7645,938.108948,26737162.00,BTCUSDT,67.369858
2023-10-17 11:00:00,2023-10-17 11:00:00,28506.130859,28527.380859,28417.550781,28422.630859,1074.049072,1697543999999,30576014.00,-20184,480.409637,13676177.00,BTCUSDT,62.100502
2023-10-17 12:00:00,2023-10-17 12:00:00,28422.630859,28471.300781,28180.000000,28218.919922,2213.095703,1697547599999,62722552.00,15982,935.105774,26500074.00,BTCUSDT,51.514469


In [5]:
data = calc_utils.calc_ema_periods(data, periods_of_time=[times_regression_PnL, 200])
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15710 entries, 2022-01-01 00:00:00 to 2023-10-17 14:00:00
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     15710 non-null  datetime64[ns]
 1   open                          15710 non-null  float32       
 2   high                          15710 non-null  float32       
 3   low                           15710 non-null  float32       
 4   close                         15710 non-null  float32       
 5   volume                        15710 non-null  float32       
 6   close_time                    15710 non-null  int64         
 7   quote_asset_volume            15710 non-null  float32       
 8   number_of_trades              15710 non-null  int16         
 9   taker_buy_base_asset_volume   15710 non-null  float32       
 10  taker_buy_quote_asset_volume  15710 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_24p,ema_24p_diff,ema_200p,ema_200p_diff
ix_open_time,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,2022-01-01 00:00:00,46216.929688,46731.390625,46208.371094,46656.128906,1503.330933,1640998799999,69879984.00,-26928,806.065369,37462156.00,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2022-01-01 01:00:00,2022-01-01 01:00:00,46656.140625,46949.988281,46574.058594,46778.140625,943.815369,1641002399999,44127148.00,31872,491.180664,22963272.00,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2022-01-01 02:00:00,2022-01-01 02:00:00,46778.140625,46928.941406,46721.960938,46811.769531,485.168610,1641005999999,22720670.00,24364,206.351135,9662607.00,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2022-01-01 03:00:00,2022-01-01 03:00:00,46811.769531,46916.628906,46760.121094,46813.199219,562.889709,1641009599999,26363260.00,19882,248.282120,11629376.00,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2022-01-01 04:00:00,2022-01-01 04:00:00,46813.210938,46887.328125,46591.230469,46711.050781,861.883911,1641013199999,40272044.00,23357,397.535645,18575402.00,BTCUSDT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-17 10:00:00,2023-10-17 10:00:00,28446.789062,28609.380859,28409.269531,28506.130859,1691.276367,1697540399999,48207104.00,7645,938.108948,26737162.00,BTCUSDT,67.369858,28207.390625,1.059085,27390.515625,4.072999
2023-10-17 11:00:00,2023-10-17 11:00:00,28506.130859,28527.380859,28417.550781,28422.630859,1074.049072,1697543999999,30576014.00,-20184,480.409637,13676177.00,BTCUSDT,62.100502,28224.609375,0.701592,27400.785156,3.729257
2023-10-17 12:00:00,2023-10-17 12:00:00,28422.630859,28471.300781,28180.000000,28218.919922,2213.095703,1697547599999,62722552.00,15982,935.105774,26500074.00,BTCUSDT,51.514469,28224.154297,-0.018546,27408.925781,2.955220


In [6]:
data = utils.regression_PnL(
    data=data,
    label=myenv.label,
    diff_percent=float(stop_loss),
    max_regression_profit_and_loss=int(times_regression_PnL),
    drop_na=True,
    drop_calc_cols=True,
    strategy=None)
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15487 entries, 2022-01-09 07:00:00 to 2023-10-16 14:00:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     15487 non-null  datetime64[ns]
 1   open                          15487 non-null  float32       
 2   high                          15487 non-null  float32       
 3   low                           15487 non-null  float32       
 4   close                         15487 non-null  float32       
 5   volume                        15487 non-null  float32       
 6   close_time                    15487 non-null  int64         
 7   quote_asset_volume            15487 non-null  float32       
 8   number_of_trades              15487 non-null  int16         
 9   taker_buy_base_asset_volume   15487 non-null  float32       
 10  taker_buy_quote_asset_volume  15487 non-null  float32      

In [7]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15487 entries, 2022-01-09 07:00:00 to 2023-10-16 14:00:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     15487 non-null  datetime64[ns]
 1   open                          15487 non-null  float32       
 2   high                          15487 non-null  float32       
 3   low                           15487 non-null  float32       
 4   close                         15487 non-null  float32       
 5   volume                        15487 non-null  float32       
 6   close_time                    15487 non-null  int64         
 7   quote_asset_volume            15487 non-null  float32       
 8   number_of_trades              15487 non-null  int16         
 9   taker_buy_base_asset_volume   15487 non-null  float32       
 10  taker_buy_quote_asset_volume  15487 non-null  float32      

In [8]:
perc_data_label = data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(data)
perc_data_label

,open_time,perc
status,,
CAI_1.0,6320,0.408084
ESTAVEL,3198,0.206496
SOBE_1.0,5969,0.385420


In [9]:
train_data = data[(data['open_time'] >= start_train_date) & (data['open_time'] < start_test_date)]
train_data = train_data.sort_values(myenv.date_features)
train_data

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_24p,ema_24p_diff,ema_200p,ema_200p_diff,status
ix_open_time,,,,,,,,,,,,,,,,,,
2022-01-09 07:00:00,2022-01-09 07:00:00,41873.621094,41908.550781,41778.949219,41791.511719,415.684753,1641715199999,17392368.0,22385,208.860855,8738995.0,BTCUSDT,50.532619,41750.910156,0.097247,43696.464844,-4.359513,SOBE_1.0
2022-01-09 08:00:00,2022-01-09 08:00:00,41791.500000,41949.988281,41615.320312,41666.671875,789.935669,1641718799999,32976732.0,29507,364.902496,15234226.0,BTCUSDT,48.228912,41744.171875,-0.185655,43673.144531,-4.594294,SOBE_1.0
2022-01-09 09:00:00,2022-01-09 09:00:00,41666.660156,41766.800781,41470.339844,41510.011719,858.204956,1641722399999,35702800.0,28971,413.392700,17199464.0,BTCUSDT,45.430012,41725.437500,-0.516294,43648.328125,-4.898965,SOBE_1.0
2022-01-09 10:00:00,2022-01-09 10:00:00,41510.000000,41728.648438,41410.000000,41644.699219,705.734680,1641725999999,29325236.0,27632,329.154144,13676652.0,BTCUSDT,48.212654,41718.980469,-0.178051,43625.378906,-4.540201,SOBE_1.0
2022-01-09 11:00:00,2022-01-09 11:00:00,41644.699219,41990.000000,41642.539062,41857.621094,783.605957,1641729599999,32785228.0,27953,443.422668,18552316.0,BTCUSDT,52.349323,41730.070312,0.305657,43605.160156,-4.007643,SOBE_1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,2022-12-31 19:00:00,16577.779297,16590.060547,16565.099609,16570.140625,4044.433594,1672516799999,67046048.0,-14485,2067.921631,34280872.0,BTCUSDT,50.101242,16570.439453,-0.001803,16683.832031,-0.681447,ESTAVEL
2022-12-31 20:00:00,2022-12-31 20:00:00,16570.140625,16574.970703,16564.089844,16568.599609,2622.143555,1672520399999,43448488.0,25169,1311.381104,21729512.0,BTCUSDT,49.712753,16570.292969,-0.010219,16682.683594,-0.683847,ESTAVEL
2022-12-31 21:00:00,2022-12-31 21:00:00,16568.189453,16571.640625,16544.119141,16548.279297,3618.773926,1672523999999,59928028.0,-26007,1746.413330,28921896.0,BTCUSDT,44.781689,16568.531250,-0.122231,16681.347656,-0.797707,ESTAVEL


In [10]:
test_data = data[data['open_time'] >= start_test_date]
test_data = test_data.sort_values(myenv.date_features)
test_data

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_24p,ema_24p_diff,ema_200p,ema_200p_diff,status
ix_open_time,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,2023-01-01 00:00:00,16541.769531,16545.699219,16508.390625,16529.669922,4364.835938,1672534799999,72146296.0,18782,2179.947754,36032352.0,BTCUSDT,42.406311,16560.267578,-0.184765,16676.904297,-0.882864,ESTAVEL
2023-01-01 01:00:00,2023-01-01 01:00:00,16529.589844,16556.800781,16525.779297,16551.470703,3590.066650,1672538399999,59376756.0,-4516,1730.249023,28617416.0,BTCUSDT,47.948830,16559.564453,-0.048877,16675.656250,-0.744712,ESTAVEL
2023-01-01 02:00:00,2023-01-01 02:00:00,16551.470703,16559.769531,16538.140625,16548.189453,3318.840332,1672541999999,54919452.0,-15674,1611.123047,26660868.0,BTCUSDT,47.212383,16558.654297,-0.063199,16674.386719,-0.756833,ESTAVEL
2023-01-01 03:00:00,2023-01-01 03:00:00,16548.189453,16548.189453,16518.210938,16533.039062,4242.080566,1672545599999,70122544.0,6652,2096.092773,34649044.0,BTCUSDT,43.862534,16556.605469,-0.142338,16672.980469,-0.839330,ESTAVEL
2023-01-01 04:00:00,2023-01-01 04:00:00,16533.039062,16535.970703,16511.919922,16521.849609,4285.009277,1672549199999,70802648.0,-1537,2188.401855,36159820.0,BTCUSDT,41.519444,16553.824219,-0.193155,16671.476562,-0.897503,ESTAVEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-16 10:00:00,2023-10-16 10:00:00,27768.060547,27788.359375,27696.220703,27770.029297,1228.591187,1697453999999,34088832.0,-18767,577.227051,16015076.0,BTCUSDT,75.465057,27356.603516,1.511247,27146.027344,2.298686,SOBE_1.0
2023-10-16 11:00:00,2023-10-16 11:00:00,27770.029297,27773.849609,27674.119141,27749.599609,924.365540,1697457599999,25636076.0,-15531,417.384491,11574208.0,BTCUSDT,73.753204,27388.042969,1.320126,27152.033203,2.200816,SOBE_1.0
2023-10-16 12:00:00,2023-10-16 12:00:00,27749.609375,27887.990234,27709.039062,27887.990234,1977.936890,1697461199999,54998668.0,12718,1031.208496,28675056.0,BTCUSDT,77.479881,27428.039062,1.676938,27159.355469,2.682813,SOBE_1.0


In [11]:
# BTCUSDT 1h best params: close,volume,quote_asset_volume,number_of_trades,rsi
# numeric_features = 'volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'.split(',')
#text_numeric_features = 'close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'
text_numeric_features = f'number_of_trades,rsi,ema_{int(times_regression_PnL)}p,ema_200p'
numeric_features = text_numeric_features.split(',')
print(f'Numeric Features: {numeric_features} - size: {len(numeric_features)}\n')

combination_numeric_features = []
if apply_combination_features:
	for size in range(1, len(numeric_features) + 1): 
		comb = map(list, combinations(numeric_features, size))	
		for c in comb:
			res = ''
			for j in c:
				res += f'{j},'
			combination_numeric_features.append(res[0:len(res) - 1])
else:
	combination_numeric_features = [text_numeric_features]

print(f'Combination Numeric Features: \n{combination_numeric_features}')

Numeric Features: ['number_of_trades', 'rsi', 'ema_24p', 'ema_200p'] - size: 4

Combination Numeric Features: 
['number_of_trades,rsi,ema_24p,ema_200p']


In [13]:
from sklearn.utils import parallel_backend
from datetime import datetime
import os
from sklearnex import patch_sklearn
from daal4py.oneapi import sycl_context
patch_sklearn()
# fix_imbalance_method: condensednearestneighbour, editednearestneighborus, repeatededitednearestneighbours, allknn, instancehardnessthreshold, nearmiss, neighbourhoodcleaningrule, onesidedselection, randomundersampler, tomeklinks, randomoversampler, smote, smotenc, smoten, adasyn, borderlinesmote, kmeanssmote, svmsmote, smoteenn, smotetomek.
# 'condensednearestneighbour,editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
# 'smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
# imbalance_list = 'editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
imbalance_list = 'instancehardnessthreshold,smoteenn,repeatededitednearestneighbours,allknn'.split(',')
simulation_results_filename = f'./resultado_simulacao_{symbol}_{interval}.csv'

df_resultado_simulacao = pd.DataFrame()
has_simulation_file = os.path.exists(simulation_results_filename)
if has_simulation_file:
  df_resultado_simulacao = pd.read_csv(simulation_results_filename, sep=';')

with sycl_context("gpu:0"):  
  with parallel_backend('threading', n_jobs=20):
    for aux_numeric_features in combination_numeric_features:
      experiement = ClassificationExperiment()
      features = []
      features += ['open_time', myenv.label]
      features += aux_numeric_features.split(',')
      print(f'features: {features}')
      for fix_imbalance_method in imbalance_list:
        if has_simulation_file:
          chave = (df_resultado_simulacao['symbol'] == symbol) & \
              (df_resultado_simulacao['interval'] == interval) & \
              (df_resultado_simulacao['estimator'] == estimator) & \
              (df_resultado_simulacao['fix_imbalance_method'] == fix_imbalance_method) & \
              (df_resultado_simulacao['features'] == str(features))

          if chave.sum() > 0:
            print(f'fix_imbalance_method: {fix_imbalance_method} already exists')
            continue

        try:
          print(f'fix_imbalance_method: {fix_imbalance_method}')
          setup = experiement.setup(
              data=train_data[features].copy(),
              train_size=myenv.train_size,
              target=myenv.label,
              numeric_features=aux_numeric_features.split(','),
              date_features=['open_time'],
              create_date_columns=["hour", "day", "month"],
              data_split_shuffle=False,
              data_split_stratify=False,
              fix_imbalance=True,
              fix_imbalance_method=fix_imbalance_method,
              remove_outliers=True,
              fold_strategy='timeseries', 
              fold=3,
              session_id=123,
              normalize=normalize,
              use_gpu=False,
              verbose=True,
              n_jobs=20,
              log_experiment=False,
          )

          if _compare_models:
            best = setup.compare_models()
            estimator = setup.pull().index[0]
            print('Estimator: ' + estimator)
          else:
            best = setup.create_model(estimator, n_jobs=20)

          if tune_model:
            best = setup.tune_model(best)

          # predict on test set
          # holdout_pred = setup.predict_model(best)
          # print('Holdout Score:', holdout_pred['prediction_score'].mean())
          # print('Holdout Score Group:\n', holdout_pred[[myenv.label, 'prediction_score']].groupby(myenv.label).mean())

          predict = setup.predict_model(best, data=test_data.drop(columns=[myenv.label]))
          predict[myenv.label] = test_data[myenv.label]
          predict['_score'] = predict['prediction_label'] == predict[myenv.label]
          # print('Predict Score Mean:', predict['_score'].mean())
          # print('Predict Score Mean Group:\n', predict[[myenv.label, '_score']].groupby(myenv.label).mean())

          final_model = setup.finalize_model(best)

          ajusted_test_data = test_data.drop(myenv.label, axis=1)
          df_final_predict, res_score = utils.validate_score_test_data(
              setup,
              final_model,
              myenv.label,
              test_data,
              ajusted_test_data)

          # df_final_predict.info()
          # print('Final Score Mean:', res_score.mean().values[0])
          # print('Final Score Group:\n', res_score)

          start_test_date = df_final_predict['open_time'].min()
          end_test_date = df_final_predict['open_time'].max()

          # print('Simule Trading:')
          # print(f'Min Data: {start_test_date}')
          # print(f'Max Data: {end_test_date}')
          saldo_inicial = 100.0
          saldo_final = utils.simule_trading_crypto2(df_final_predict, start_test_date, end_test_date, saldo_inicial, stop_loss)
          print(f'>>>> Saldo Final: {saldo_final} - features: {features}\n\n')

          result_simulado = {}
          result_simulado['date'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
          result_simulado['symbol'] = symbol
          result_simulado['interval'] = interval
          result_simulado['estimator'] = estimator
          result_simulado['fix_imbalance_method'] = fix_imbalance_method
          result_simulado['stop_loss'] = stop_loss
          result_simulado['times_regression_profit_and_loss'] = times_regression_PnL
          result_simulado['features'] = features
          result_simulado['final_value'] = saldo_final

          if res_score is not None:
            result_simulado['score'] = ''
            for i in range(0, len(res_score.index.values)):
              result_simulado['score'] += f'[{res_score.index.values[i]}={res_score["_score"].values[i]:.2f}]'

          df_resultado_simulacao = pd.concat([df_resultado_simulacao, pd.DataFrame([result_simulado])], ignore_index=True)
          df_resultado_simulacao.sort_values('final_value', inplace=True)

          df_resultado_simulacao.to_csv(simulation_results_filename, sep=';', index=False)
        except Exception as e:
          print(e)
          continue

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


RuntimeError: Device is not supported: gpu:0